In [1]:
#Importing Libraries
import warnings
warnings.filterwarnings("ignore")
import csv
import pandas as pd
import datetime 
import time 
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns
from matplotlib import rcParams  
from sklearn.cluster import MiniBatchKMeans, KMeans
import math
import pickle
import os
import xgboost as xgb
import warnings
import networkx as nx
import pdb
import pickle
from pandas import HDFStore,DataFrame
from pandas import read_hdf
from scipy.sparse.linalg import svds, eigs
import gc
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
#Load the graph
if os.path.isfile('data/after_eda/train_pos_after_eda.csv'):
    train_graph=nx.read_edgelist('data/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
    
    print(nx.info(train_graph))
else:
    print("Please run the FB_EDA.ipynb or download the files from drive")

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


In [3]:
#reading
from pandas import read_hdf
df_final_train = read_hdf('data/fea_sample/storage_sample_stage4.h5', 'train_df',mode='r')
df_final_test = read_hdf('data/fea_sample/storage_sample_stage4.h5', 'test_df',mode='r')

In [4]:
#Get feature information
df_final_train.columns

Index(['source_node', 'destination_node', 'indicator_link',
       'jaccard_followers', 'jaccard_followees', 'cosine_followers',
       'cosine_followees', 'num_followers_s', 'num_followees_s',
       'num_followees_d', 'inter_followers', 'inter_followees', 'adar_index',
       'follows_back', 'same_comp', 'shortest_path', 'weight_in', 'weight_out',
       'weight_f1', 'weight_f2', 'weight_f3', 'weight_f4', 'page_rank_s',
       'page_rank_d', 'katz_s', 'katz_d', 'hubs_s', 'hubs_d', 'authorities_s',
       'authorities_d', 'svd_u_s_1', 'svd_u_s_2', 'svd_u_s_3', 'svd_u_s_4',
       'svd_u_s_5', 'svd_u_s_6', 'svd_u_d_1', 'svd_u_d_2', 'svd_u_d_3',
       'svd_u_d_4', 'svd_u_d_5', 'svd_u_d_6', 'svd_v_s_1', 'svd_v_s_2',
       'svd_v_s_3', 'svd_v_s_4', 'svd_v_s_5', 'svd_v_s_6', 'svd_v_d_1',
       'svd_v_d_2', 'svd_v_d_3', 'svd_v_d_4', 'svd_v_d_5', 'svd_v_d_6'],
      dtype='object')

## 4.5 Preferential Attachment:

1. https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_prediction.preferential_attachment.html
2. http://be.amazd.com/link-prediction/
3. D. Liben-Nowell, J. Kleinberg. The Link Prediction Problem for Social Networks (2004). http://www.cs.cornell.edu/home/kleinber/link-pred.pdf

<b>Preferential Attachment</b>: One well-known concept in social networks is that users with many friends tend to create more connections in the future. This is due to the fact that in some social networks, like in finance, the rich get richer. We estimate how "rich" our two vertices are by calculating the multiplication between the number of friends (|Γ(x)|) or followers each vertex has. It may be noted that the similarity index does not require any node neighbor information; therefore, this similarity index has the lowest computational complexity. 

In [5]:
#Preferential attachment for followees
def prefer_attach_followees(a,b):
    try:
        if (len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0):
            return 0
        else:
            pa_score = len(set(train_graph.successors(a))) * len(set(train_graph.successors(b)))
    except:
        return 0
    return pa_score

In [6]:
prefer_attach_followees(832016, 1543415)

8662

In [7]:
#Preferential attachment for followees
def prefer_attach_followers(a,b):
    try:
        if (len(set(train_graph.predecessors(a))) == 0  | len(set(train_graph.predecessors(b))) == 0):
            return 0
        else:
            pa_score = len(set(train_graph.predecessors(a))) * len(set(train_graph.predecessors(b)))
    except:
        return 0
    return pa_score

In [8]:
prefer_attach_followers(832016, 1543415)

1598

### Adding the new features to the dataset

In [9]:
#Adding features to train data
df_final_train['pref_attach_followees'] = df_final_train.apply(lambda row: prefer_attach_followees(row['source_node'],row['destination_node']),axis=1)
df_final_train['pref_attach_followers'] = df_final_train.apply(lambda row: prefer_attach_followers(row['source_node'],row['destination_node']),axis=1)

In [10]:
#Adding features to test data
df_final_test['pref_attach_followees'] = df_final_test.apply(lambda row: prefer_attach_followees(row['source_node'],row['destination_node']),axis=1)
df_final_test['pref_attach_followers'] = df_final_test.apply(lambda row: prefer_attach_followers(row['source_node'],row['destination_node']),axis=1)

In [13]:
df_final_train[['pref_attach_followees','pref_attach_followers']].head()

,pref_attach_followees,pref_attach_followers
0,120,66
1,8662,1598
2,902,980
3,35,22
4,33,5


In [14]:
df_final_test[['pref_attach_followees','pref_attach_followers']].head()

,pref_attach_followees,pref_attach_followers
0,54,84
1,19,34
2,144,150
3,340,407
4,405,324
